In [112]:
import numpy as np
import pandas as pd
# import sys
import re
import time
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
model_path = '/home/lambert/Git/Titanic/Titanic_model/'

In [113]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [114]:
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [115]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [116]:
# 1. Data clearning
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [117]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']
LABEL = ['Survived', 'Deceased']

In [118]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = df_train['Cabin'].unique()
X_lack = df_test.Cabin[df_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(df_train['Embarked'].tolist()))

df_train.loc[:,'Cabin'] = le_Cabin.transform(df_train['Cabin'])
df_train.loc[:,'Embarked'] = le_Embarked.transform(df_train['Embarked'])
df_test.loc[:,'Cabin'] = le_Cabin.transform(df_test['Cabin'])
df_test.loc[:,'Embarked'] = le_Embarked.transform(df_test['Embarked'])

In [124]:
# Do the feature engineer in DF, keep train and label index consistance
# Split Validation with the selected features
# X_train, X_val, y_train, y_val = train_test_split(X, y)
X_train, X_valid, y_train, y_valid = train_test_split(df_train[sFeatures], df_train['Survived'])

In [120]:
# No Validation
# X_train = df_train[sFeatures]
# y_train = df_train['Survived']

### Features Input

In [134]:
def make_input_fn(X, y, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = X,
    y = y,
    batch_size = 10,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [135]:
def make_prediction_input_fn(X, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = X,
    batch_size = 10,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [136]:
def make_feature_cols(X):
  input_columns = [tf.feature_column.numeric_column(k) for k in X.columns]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [138]:
X_train.shape

(666, 10)

In [139]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR_LR = 'Titanic_trained_LR'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model_LR = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(X_train), model_dir = OUTDIR)

model_LR.train(input_fn = make_input_fn(X_train, y_train, num_epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Titanic_trained_LR', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x118030e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into Titanic_trained_LR/model.ckpt.
INFO:tensorflow:loss = 1.0, step = 1
INFO:tensorflow:g

<h3> Validate the model </h3>

In [140]:
def print_rmse(model, name, X, y):
  metrics = model.evaluate(input_fn = make_input_fn(X, y, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))

In [141]:
print_rmse(model_LR, 'validation', X_valid, y_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-29-04:15:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Titanic_trained_LR/model.ckpt-6660
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-29-04:15:53
INFO:tensorflow:Saving dict for global step 6660: average_loss = 0.16010863, global_step = 6660, loss = 1.5523576
RMSE on validation dataset = 0.40013575553894043


<h3> Deep Neural Network regression </h3>

In [143]:
OUTDIR_DNN = 'Titanic_trained_DNN'
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR_DNN, ignore_errors = True) # start fresh each time
model_DNN = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(X_train), model_dir = OUTDIR_DNN)
model_DNN.train(input_fn = make_input_fn(X_train, y_train, num_epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Titanic_trained_DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1170558d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into Titanic_trained_DNN/model.ckpt.
INFO:tensorflow:loss = 5.0, step = 1
INFO:tensorflow

In [145]:
print_rmse(model_DNN, 'validation', X_valid, y_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-29-04:24:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Titanic_trained_DNN/model.ckpt-6660
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-29-04:24:22
INFO:tensorflow:Saving dict for global step 6660: average_loss = 0.23695736, global_step = 6660, loss = 2.2974563
RMSE on validation dataset = 0.48678267002105713


In [48]:
# Tensorflow variables
# with tf.device('/device:GPU:0'):
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [49]:
# Cross Entropy
# with tf.device('/device:GPU:0'):
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [50]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [51]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [52]:
# sess.close()

In [53]:
# training starting option 1 with CPU
# for epoch in range(100):
#     total_loss = 0.
#     for i in range(len(X_train)):
#         feed = {X_place: X_train, y_place: y_train}
#         _, loss = sess.run([train_op, cost], feed_dict=feed)
#         total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [ ]:
# training starting option 1 with GPU
# with tf.device('/device:GPU:0'):
start = time.time()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device('/device:GPU:0'):
        sess.run(tf.global_variables_initializer())
        for epoch in range(10000):
            total_loss = 0.
            for i in range(len(X_train)):
                feed = {X_place: X_train, y_place: y_train}
                _, loss = sess.run([train_op, cost], feed_dict=feed)
                total_loss += loss
            print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
        #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
        #     print('predict: {:.6f}'.format(y_pred))
        print ('Training complete!')

        feed_dict_train = {X_place: X_train}
        prediction = sess.run(y_pred, feed_dict=feed_dict_train)
        correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
        # evaluate the accuracy
        accuracy = np.mean(correct)
        print('Accuracy on validation: {:2.2%}'.format (accuracy))
    save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
end = time.time()
print('time spend: {:.4f} sec'.format(end-start))

Epoch: 0001, total loss=7741.516098022
Epoch: 0002, total loss=7738.829321861
Epoch: 0003, total loss=7734.809428215
Epoch: 0004, total loss=7731.230428696
Epoch: 0005, total loss=7730.431954384
Epoch: 0006, total loss=7730.153929710
Epoch: 0007, total loss=7729.909674644
Epoch: 0008, total loss=7729.665932655
Epoch: 0009, total loss=7729.419364929
Epoch: 0010, total loss=7729.170191765
Epoch: 0011, total loss=7728.918432236
Epoch: 0012, total loss=7728.664295197
Epoch: 0013, total loss=7728.407405853
Epoch: 0014, total loss=7728.149026871
Epoch: 0015, total loss=7727.888969421
Epoch: 0016, total loss=7727.626473427
Epoch: 0017, total loss=7727.363004684
Epoch: 0018, total loss=7727.097887993
Epoch: 0019, total loss=7726.830873489
Epoch: 0020, total loss=7726.563403130
Epoch: 0021, total loss=7726.293632507
Epoch: 0022, total loss=7726.022218704
Epoch: 0023, total loss=7725.749572754
Epoch: 0024, total loss=7725.473988533
Epoch: 0025, total loss=7725.196638107
Epoch: 0026, total loss=7

In [ ]:
# save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [21]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 81.98%


In [22]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 74.89%


In [30]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
    y_predict_1 = np.argmin(prediction, axis=1)
    
# evaluate the accuracy

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt


In [32]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict_1}).set_index('PassengerId')
submission.to_csv('submission_tf_GDO_v3.csv')

In [29]:
y_predict

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,

In [31]:
y_predict_1

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [27]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
# print_accuracy()
# plot_example_errors()
# plot_weights()

In [ ]:
File_Writer=tf.summary.FileWriter(''/Users/lambert/Documents/Python_code/tensorflow_code/Titanic'
                                  ,sess.graph)

In [ ]:
# with tf.device('/device:GPU:0'):
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:

In [ ]:
# allocate as much GPU memory based on runtime allocations
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
# allocate 40% memory of GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config, ...)

In [ ]:
# Automatically choose existing in case specified one doesn't exit
# allow_soft_placement=True
with tf.device('/device:GPU:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))

In [ ]:
sess.close()

In [ ]:
sys.argv[2]

In [1]:
device_lib.list_local_devices()

NameError: name 'device_lib' is not defined